# importing libraries and data reading

In [1]:
import pandas as pd
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)
pd.set_option('display.max_colwidth',None)
import re
import ast
import numpy as np
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import warnings
warnings.filterwarnings("ignore")

In [178]:
df = pd.read_excel ('Tier A.xlsx')

In [181]:
a=['countyId','zipcode', 'zestimate','lotAreaValue',
       'rentZestimate','bedrooms', 'bathrooms', 'livingAreaValue', 
       'lastSoldPrice', 'zestimateLowPercent', 'zestimateHighPercent',
       'restimateLowPercent', 'restimateHighPercent', 
       'taxAssessedValue', 'taxAssessedYear', 
       'propertyTaxRate',  
       'resoFacts_hasFireplace', 'resoFacts_parking',
       'mortgageRates_thirtyYearFixedRate',
       'mortgageRates_fifteenYearFixedRate', 'mortgageRates_arm5Rate', 
    'city', 'county' , 'description','parentRegion_name',
    'dateSoldString', 'resoFacts_daysOnZillow', 'resoFacts_flooring', 'resoFacts_hasAttachedProperty',
    'resoFacts_hasGarage', 'resoFacts_hasPetsAllowed', 'resoFacts_hasSpa' , 'resoFacts_hasView','resoFacts_hoaFee',
     'resoFacts_pricePerSquareFoot','resoFacts_yearBuilt' ,'solarPotential_sunScore','schools'
   ]

In [182]:
df=df[a]

In [183]:
df.shape

(742, 38)

In [184]:
df.isnull().sum().sum() # total null values in our dataset

2782

# Data Cleaning and Null Values removal

In [185]:
df['resoFacts_hoaFee'].fillna(df['resoFacts_hoaFee'].mode()[0], inplace=True)
hoe=[]
for x in df['resoFacts_hoaFee']:
    x=str(x).replace(",","")
    hoe.append(int(re.findall('[0-9,]+', x)[0]))
df['resoFacts_hoaFee']=hoe   

In [186]:
df['resoFacts_flooring'].fillna(df['resoFacts_flooring'].mode()[0], inplace=True)
flooring=[]
for x in df['resoFacts_flooring']:
    x=x.replace('[',"")
    flooring.append(x.replace(']',""))
df['resoFacts_flooring']= flooring

In [187]:
df_cat= df.select_dtypes(exclude='number')
print(df_cat.shape)

(742, 6)


In [188]:
print(df_cat.isnull().sum())

city                  0
county                0
description           2
parentRegion_name     2
resoFacts_flooring    0
schools               4
dtype: int64


In [189]:
for column in df_cat:
    df_cat[column].fillna(df_cat[column].mode()[0], inplace=True)

In [191]:
discrete_cols= ["bedrooms", "bathrooms", "resoFacts_hasFireplace", "resoFacts_parking", "resoFacts_daysOnZillow" ,"resoFacts_hasAttachedProperty" , "resoFacts_hasGarage" , "resoFacts_hasPetsAllowed", "resoFacts_hasSpa", "resoFacts_hasView", "resoFacts_yearBuilt"]

In [192]:
numeric_cols=["lotAreaValue","zestimate","rentZestimate", "livingAreaValue", "lastSoldPrice", "zestimateLowPercent", "zestimateHighPercent", "restimateLowPercent" ,"restimateHighPercent" , "taxAssessedValue" , "propertyTaxRate", "resoFacts_hoaFee", "resoFacts_pricePerSquareFoot" , "solarPotential_sunScore"]

In [193]:
df_discrete=df[discrete_cols]
df_num=df[numeric_cols]

In [194]:
df_num.isnull().sum()

lotAreaValue                     58
zestimate                       169
rentZestimate                   122
livingAreaValue                   9
lastSoldPrice                   222
zestimateLowPercent             169
zestimateHighPercent            169
restimateLowPercent             177
restimateHighPercent            177
taxAssessedValue                134
propertyTaxRate                   2
resoFacts_hoaFee                  0
resoFacts_pricePerSquareFoot     37
solarPotential_sunScore         146
dtype: int64

In [195]:
from sklearn.impute import KNNImputer   # missing values imputation

imputer = KNNImputer(n_neighbors=2)  #impute missing values
df_num.iloc[:,0:]=imputer.fit_transform(df_num.iloc[:,0:]) #replace df with imputed missing values

In [196]:
df_num.isnull().sum().sum()

0

In [197]:
df_discrete.isnull().sum()

bedrooms                          22
bathrooms                          8
resoFacts_hasFireplace           117
resoFacts_parking                  0
resoFacts_daysOnZillow             1
resoFacts_hasAttachedProperty      0
resoFacts_hasGarage                0
resoFacts_hasPetsAllowed           0
resoFacts_hasSpa                   0
resoFacts_hasView                  0
resoFacts_yearBuilt               10
dtype: int64

In [198]:
#df_discret= df_discrete.dropna()

In [199]:
df_discrete.resoFacts_hasFireplace.value_counts()

1.00000    385
0.00000    240
Name: resoFacts_hasFireplace, dtype: int64

In [200]:
df_discrete['resoFacts_hasFireplace'].fillna(0, inplace=True)

In [201]:
df_discrete.isnull().sum()

bedrooms                         22
bathrooms                         8
resoFacts_hasFireplace            0
resoFacts_parking                 0
resoFacts_daysOnZillow            1
resoFacts_hasAttachedProperty     0
resoFacts_hasGarage               0
resoFacts_hasPetsAllowed          0
resoFacts_hasSpa                  0
resoFacts_hasView                 0
resoFacts_yearBuilt              10
dtype: int64

In [202]:
for column in df_discrete:
    df_discrete[column].fillna(df_discrete[column].mode()[0], inplace=True)

In [203]:
df_discrete.isnull().sum()

bedrooms                         0
bathrooms                        0
resoFacts_hasFireplace           0
resoFacts_parking                0
resoFacts_daysOnZillow           0
resoFacts_hasAttachedProperty    0
resoFacts_hasGarage              0
resoFacts_hasPetsAllowed         0
resoFacts_hasSpa                 0
resoFacts_hasView                0
resoFacts_yearBuilt              0
dtype: int64

In [204]:
print(df_cat.shape)
print(df_num.shape)
print(df_discrete.shape)

(742, 6)
(742, 14)
(742, 11)


In [205]:
df_discrete['resoFacts_parking']=np.where(df_discrete['resoFacts_parking']>=1,1,0) 


In [206]:
df_discrete.resoFacts_parking.value_counts()

1    512
0    230
Name: resoFacts_parking, dtype: int64

In [207]:
df= pd.concat([df_cat,df_num,df_discrete], axis=1)  # this df is without null values

In [208]:
df.shape

(742, 31)

In [209]:
df.isnull().sum().sum()

0

# Feature Engineering- Creating 6 features out of school

In [210]:
#df_2= pd.read_excel('zillow_full.xlsx')

In [211]:
schools= list(df['schools'])

In [212]:
list_schools= [[]] * len(schools)

In [213]:
len(list_schools)

742

In [214]:
i=0
for x in schools:
    list_schools[i]=[0,0,0,0,0,0]
    x = str(x).replace('\\','"')
    l=re.findall("[{][\\a-zA-z0-9:',\-/ \.=?&]+[}]", x)
    for y in l:
        y=ast.literal_eval(y)
        if y['level']== 'Elementary':
            if y['rating']!=None:
                if list_schools[i][1]<y['rating']:
                    list_schools[i][1]=y['rating']
                    list_schools[i][0]=y['distance']
                
        if y['level']== 'Middle':
            if y['rating']!=None:
                if list_schools[i][3]<y['rating']:
                    list_schools[i][3]=y['rating']
                    list_schools[i][2]=y['distance']
        if y['level']=='High':
            if y['rating']!=None:
                if list_schools[i][5]<y['rating']:
                    list_schools[i][5]=y['rating']
                    list_schools[i][4]=y['distance']
    
    i+=1
    
    

In [215]:
list_schools[0]

[0.4, 8, 1.4, 6, 1.3, 7]

In [216]:
for i in range(len(list_schools)):
    list_schools[i] = [None if item <=0 else item for item in list_schools[i]]

In [217]:
df_3 = pd.DataFrame(list_schools, columns = ['Elementary_Distance', 'Elementary_Ratings', 'Middle_Distance', 'Middle_Ratings', 'High_Distance', 'High_Ratings'])

In [218]:
df_3['Elementary_Distance']=np.where(np.isnan(df_3['Elementary_Distance']),df_3['Elementary_Distance'].median(),df_3['Elementary_Distance']) 
df_3['Middle_Ratings']=np.where(np.isnan(df_3['Middle_Ratings']),df_3['Middle_Ratings'].mode(),df_3['Middle_Ratings']) 
df_3['Middle_Distance']=np.where(np.isnan(df_3['Middle_Distance']),df_3['Middle_Distance'].median(),df_3['Middle_Distance']) 
df_3['High_Ratings']=np.where(np.isnan(df_3['High_Ratings']),df_3['High_Ratings'].mode(),df_3['High_Ratings']) 
df_3['High_Distance']=np.where(np.isnan(df_3['High_Distance']),df_3['High_Distance'].median(),df_3['High_Distance']) 


In [219]:
imputer = KNNImputer(n_neighbors=2)  #impute missing values
df_3.iloc[:,0:]=imputer.fit_transform(df_3.iloc[:,0:]) #replace df with imputed missing values

In [220]:
df_3.shape

(742, 6)

In [221]:
df_3.isnull().sum()

Elementary_Distance    0
Elementary_Ratings     0
Middle_Distance        0
Middle_Ratings         0
High_Distance          0
High_Ratings           0
dtype: int64

In [222]:
df.shape

(742, 31)

In [223]:
df.drop('schools', axis=1, inplace= True)

In [224]:
df.shape

(742, 30)

In [225]:
df=df.reset_index()

In [226]:
df=pd.concat([df,df_3], axis=1)

In [227]:
df.shape  # this df contains school columns also

(742, 37)

In [228]:
df.isnull().sum().sum()

0

# Outliers

In [229]:
#columns_for_outliers=[ 'bedrooms','lastSoldPrice','Elementary_Ratings','Elementary_Distance',
 #      'Middle_Ratings','Middle_Distance', 'High_Distance', 'High_Ratings','livingAreaValue','resoFacts_hasGarage']

In [230]:
columns_for_outliers= ['Elementary_Ratings','Elementary_Distance',
    'Middle_Ratings','Middle_Distance', 'High_Distance', 'High_Ratings']

In [231]:
# Detecting outliers and imputing nan 
from scipy.stats import skew

for x in columns_for_outliers:
        percentile25 = df[x].quantile(0.25)
        percentile75 = df[x].quantile(0.75)
        iqr= percentile75-percentile25
        upper = percentile75 + 1.5 * iqr
        lower = percentile25 - 1.5 * iqr
        for y in df[x]:
            if (y>upper) or (y<lower):
                df[x].loc[df[x] == y]=None
    
    
       

In [232]:
df.isnull().sum()

index                             0
city                              0
county                            0
description                       0
parentRegion_name                 0
resoFacts_flooring                0
lotAreaValue                      0
zestimate                         0
rentZestimate                     0
livingAreaValue                   0
lastSoldPrice                     0
zestimateLowPercent               0
zestimateHighPercent              0
restimateLowPercent               0
restimateHighPercent              0
taxAssessedValue                  0
propertyTaxRate                   0
resoFacts_hoaFee                  0
resoFacts_pricePerSquareFoot      0
solarPotential_sunScore           0
bedrooms                          0
bathrooms                         0
resoFacts_hasFireplace            0
resoFacts_parking                 0
resoFacts_daysOnZillow            0
resoFacts_hasAttachedProperty     0
resoFacts_hasGarage               0
resoFacts_hasPetsAllowed    

In [233]:
df= df.dropna()

In [234]:
df.shape

(615, 37)

# Feature Engineering- Creating New feature out of feature("description")

In [235]:
sentiment_des = []
from textblob import TextBlob
for text in df['description'].values:
        t= TextBlob(text)
        sentiment_des.append(t.sentiment[0])
df['desc_sentiment']=sentiment_des

In [236]:
df['desc_sentiment']=sentiment_des

In [237]:
df.drop(['description'], axis=1,inplace=True)

In [238]:
df.shape 

(615, 37)

In [239]:
df.corr()

,index,lotAreaValue,zestimate,rentZestimate,livingAreaValue,lastSoldPrice,zestimateLowPercent,zestimateHighPercent,restimateLowPercent,restimateHighPercent,taxAssessedValue,propertyTaxRate,resoFacts_hoaFee,resoFacts_pricePerSquareFoot,solarPotential_sunScore,bedrooms,bathrooms,resoFacts_hasFireplace,resoFacts_parking,resoFacts_daysOnZillow,resoFacts_hasAttachedProperty,resoFacts_hasGarage,resoFacts_hasPetsAllowed,resoFacts_hasSpa,resoFacts_hasView,resoFacts_yearBuilt,Elementary_Distance,Elementary_Ratings,Middle_Distance,Middle_Ratings,High_Distance,High_Ratings,desc_sentiment
index,1.00000,0.21948,0.01264,-0.08345,-0.00506,-0.24461,-0.33418,-0.31079,-0.05339,-0.05535,-0.03201,0.33380,-0.10221,0.05813,0.04461,0.11524,0.05915,0.14161,-0.13439,0.20531,-0.25050,-0.12101,-0.13712,0.05029,-0.08867,0.10775,0.03985,0.01854,-0.00945,0.11634,0.20705,0.18765,-0.00337
lotAreaValue,0.21948,1.00000,0.05799,-0.06329,0.01414,-0.00319,-0.09365,-0.11211,-0.10863,-0.06112,-0.10545,0.25312,-0.12822,0.01951,0.08667,0.21722,0.04403,0.30488,-0.00683,0.04097,-0.22145,-0.00268,-0.14460,0.00089,-0.09359,0.11111,-0.07598,0.10856,-0.08738,0.10692,-0.02418,0.11773,0.01472
zestimate,0.01264,0.05799,1.00000,0.84205,0.54567,0.47359,0.10555,0.08796,0.34424,0.24225,0.10323,-0.19619,0.20678,-0.00731,-0.01669,0.45614,0.41658,0.14511,0.07090,0.08228,-0.03403,0.08084,-0.04387,-0.06930,0.19517,0.00458,-0.07354,0.14662,-0.08457,0.16491,-0.08276,0.04144,0.07709
rentZestimate,-0.08345,-0.06329,0.84205,1.00000,0.44384,0.41226,0.08923,0.08806,0.41488,0.22331,0.12024,-0.30243,0.14547,-0.03702,-0.02874,0.37020,0.39325,0.07307,0.13213,-0.00261,0.06070,0.13081,-0.02821,-0.03721,0.20703,-0.06079,-0.03132,0.09630,-0.09423,0.08041,-0.15074,-0.03252,0.09124
livingAreaValue,-0.00506,0.01414,0.54567,0.44384,1.00000,0.50960,0.19865,0.21308,0.20164,0.19680,0.10683,-0.07024,-0.03562,-0.06263,-0.09618,0.59065,0.51550,0.08141,0.08867,0.06099,-0.20509,0.13158,-0.09999,0.00485,0.14320,-0.00092,0.03811,0.08179,-0.04624,0.17356,0.01520,0.07921,0.05430
lastSoldPrice,-0.24461,-0.00319,0.47359,0.41226,0.50960,1.00000,0.23650,0.27422,0.22236,0.17105,0.18584,-0.21329,0.06158,-0.00086,-0.02642,0.22438,0.29900,-0.03734,0.04699,-0.04513,-0.00813,0.09095,0.05784,-0.05640,0.15928,-0.04788,-0.03947,0.06332,0.00027,0.05283,-0.05383,-0.09357,0.04580
zestimateLowPercent,-0.33418,-0.09365,0.10555,0.08923,0.19865,0.23650,1.00000,0.90867,0.16484,0.22067,0.00836,-0.24493,-0.05321,0.03735,-0.05485,0.05806,-0.08072,-0.13462,-0.01173,-0.07895,0.07243,0.04408,-0.05725,-0.08469,-0.04807,-0.10690,-0.10198,-0.05525,-0.07435,0.00607,-0.19317,-0.16994,0.03111
zestimateHighPercent,-0.31079,-0.11211,0.08796,0.08806,0.21308,0.27422,0.90867,1.00000,0.14569,0.24232,0.06172,-0.22420,-0.05081,0.03058,-0.03891,0.03033,-0.08970,-0.16226,-0.02613,-0.04060,0.03266,0.01976,-0.03833,-0.06824,-0.01321,-0.10388,-0.06301,-0.10241,-0.04340,-0.04848,-0.14918,-0.20094,-0.02426
restimateLowPercent,-0.05339,-0.10863,0.34424,0.41488,0.20164,0.22236,0.16484,0.14569,1.00000,0.66853,0.12514,-0.26150,0.05536,-0.04546,-0.02399,0.15137,0.11607,-0.01559,0.06042,-0.07737,0.12479,0.06165,0.02561,-0.05972,0.09065,-0.06276,-0.01390,-0.01847,-0.13244,-0.04265,-0.13473,-0.06618,0.04388
restimateHighPercent,-0.05535,-0.06112,0.24225,0.22331,0.19680,0.17105,0.22067,0.24232,0.66853,1.00000,-0.00101,-0.25060,0.14013,-0.02269,-0.05704,0.04320,-0.01648,-0.11208,0.03428,-0.06064,0.07661,0.01202,0.05481,-0.05329,0.06497,-0.08331,-0.01428,-0.09713,-0.12511,-0.03584,-0.11994,-0.11260,0.04893


# Train-Test Split

In [240]:
y = df.zestimate

In [241]:
x= df.drop('zestimate', axis=1)

In [242]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.2, random_state=123)

In [243]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(492, 36)
(492,)
(123, 36)
(123,)


# Numeric Data Preparation

In [286]:
from scipy.sparse import coo_matrix

X_train_num_processed = coo_matrix((492,0))
X_test_num_processed = coo_matrix((123,0))

In [287]:
num_cols= [ 'bedrooms','rentZestimate','lastSoldPrice','resoFacts_hasFireplace' ]

In [288]:
for x in num_cols:
    

    s_train = (X_train[x].values.reshape(-1, 1))

    s_test = (X_test[x].values.reshape(-1, 1))

    
    X_train_num_processed= hstack((X_train_num_processed,s_train)).tocsr()
    X_test_num_processed= hstack((X_test_num_processed,s_test)).tocsr()


In [289]:
print(X_train_num_processed.shape)
print(X_test_num_processed.shape)

(492, 4)
(123, 4)


# OLS regresssion 

In [290]:
X_train_test_processed=pd.concat([pd.DataFrame(X_train_num_processed.todense()),pd.DataFrame(X_test_num_processed.todense())],ignore_index=True)
Y_train_test_processed = pd.concat([pd.DataFrame(Y_train),pd.DataFrame(Y_test)], ignore_index=True)

In [291]:
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats


X2 = sm.add_constant(X_train_test_processed)
est = sm.OLS(Y_train_test_processed, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:              zestimate   R-squared:                       0.753
Model:                            OLS   Adj. R-squared:                  0.751
Method:                 Least Squares   F-statistic:                     465.0
Date:                Thu, 15 Jun 2023   Prob (F-statistic):          1.28e-183
Time:                        19:04:49   Log-Likelihood:                -8920.4
No. Observations:                 615   AIC:                         1.785e+04
Df Residuals:                     610   BIC:                         1.787e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -2.43e+05   5.75e+04     -4.227      0.0

# Interpretation

1.
const (Intercept): The constant term is -2.43e+05 or -243,000. This can be interpreted as the baseline value of `zestimate` when all the predictor variables are zero. However, interpreting the intercept might not make sense in this context since it's not meaningful to have 0 bedrooms, 0 rentZestimate, etc.

2.
bedrooms (0): The coefficient for bedrooms is 1.046e+05 or 104,600. This means that, on average, for each additional bedroom, the `zestimate` is expected to increase by approximately 104,600, holding other variables constant. The p-value is close to 0, indicating that the number of bedrooms is a statistically significant predictor of the `zestimate`.

3.
rentZestimate (1): The coefficient for rentZestimate is 287.4495, or approximately 287.45. This means that, on average, for each additional dollar in the rentZestimate, the `zestimate` is expected to increase by approximately 287.45, holding other variables constant. The p-value is close to 0, which indicates that rentZestimate is a statistically significant predictor.

4.
lastSoldPrice (2): The coefficient for lastSoldPrice is 0.1722. This implies that for each additional dollar in the lastSoldPrice, the `zestimate` is expected to increase by 0.1722, holding other variables constant. The p-value is also close to 0, indicating that lastSoldPrice is a statistically significant predictor.

5.
resoFacts_hasFireplace (3: The coefficient for resoFacts_hasFireplace is 1.269e+05 or 126,900. Since resoFacts_hasFireplace is likely a binary variable (indicating the presence or absence of a fireplace), this coefficient can be interpreted as the average change in `zestimate` when a property has a fireplace compared to not having one, holding other variables constant. The p-value is small (0.002), indicating that having a fireplace is a statistically significant predictor of the `zestimate`.
